<div>
<img src="pose_landmarks_index.png" width="400"/>
</div>

In [12]:
connections_of_interest = [
    (0, 2),   # nose, left eye
    (0, 5),   # nose, right eye
    (0, 7),   # nose, left ear
    (0, 8),   # nose, right ear
    (0, 9),   # nose, left mouth
    (0, 10),  # nose, right mouth
    (2, 5),   # left eye, right eye
    (2, 7),   # left eye, left ear
    (5, 8),   # right eye, right ear
    (7, 9),   # left ear, left mouth
    (7, 11),  # Left ear, left shoulder
    (8, 10),  # right ear, right mouth
    (8, 12),  # right ear, right shoulder
    (9, 10),  # left mouth, right mouth
    (9, 11),  # left mouth, left shoulder
    (10, 12), # right mout, right shoulder
    (11, 12), # left shoulder, right shoulder
    (12,10),  # Right shoulder, right mouth
]

connections_descriptions = {
    (0, 2): "nose, left eye",
    (0, 5): "nose, right eye",
    (0, 7): "nose, left ear",
    (0, 8): "nose, right ear",
    (0, 9): "nose, left mouth",
    (0, 10): "nose, right mouth",
    (2, 5): "left eye, right eye",
    (2, 7): "left eye, left ear",
    (5, 8): "right eye, right ear",
    (7, 9): "left ear, left mouth",
    (7, 11): "Left ear, left shoulder",
    (8, 10): "right ear, right mouth",
    (8, 12): "right ear, right shoulder",
    (9, 10): "left mouth, right mouth",
    (9, 11): "left mouth, left shoulder",
    (10, 12): "right mout, right shoulder",
    (11, 12): "left shoulder, right shoulder",
    (12, 10): "right shoulder, right mouth"}



In [13]:
import numpy as np
import mediapipe as mp
import random
import math

from pathlib import Path
from typing import List, Dict

In [14]:
project_data = Path(r"./ml_assignment_data")

#actions that we try to detect
actions = ['C','TL','TR','BL','BR']
actions_description = {'C':'MIDDEN','TL':'LINKS BOVEN','TR':'RECHTS BOVEN','BL':'LINKS ONDER','BR':'RECHTS ONDER' }
label_map = {label:num for num, label in enumerate(actions)}

In [19]:
def read_file(np_file):
    # read a flattened file and reshape and return only x and y
    flat_file = np.load(np_file)
    num_landmarks = len(flat_file) // 4
    return flat_file.reshape(num_landmarks, 4)

file= Path(r"C:\Users\djase\Documents\Projecten\ML\ml_assignment_data\person_0000\BL\0\0.npy")
assert(read_file(str(file)).shape == (33,4))

In [26]:
def calculate_distance(results):
    distances =[]
    for connection in connections_of_interest:
        
        sx, sy, _, _ = tuple(results[connection[0]])
        ex, ey, _, _ = tuple(results[connection[1]])
        distances.append( math.sqrt(math.pow(sx-ex,2) + math.pow(sy - ey,2)))

    return distances
        
file= Path(r"C:\Users\djase\Documents\Projecten\ML\ml_assignment_data\person_0000\BL\0\0.npy")
data = read_file(str(file))
dist = calculate_distance(data)
assert (len(dist)==len(connections_descriptions)==len(connections_of_interest))
print(len(dist))

18


In [32]:
def read_data(actions: List)-> Dict:
    person_folders = [x for x in project_data.iterdir() if x.is_dir() and str(x.name).split('_')[-2] == 'person']
    samples= [] 
    labels =[]
    pers_id=[]
    global series
    global frames
    p_id = 0
    for person_folder in person_folders:
        for action in actions:
            action_folder = person_folder / action
            series = [x for x in action_folder.iterdir() if x.is_dir()]
            for serie in series:
                frames = [x for x in serie.iterdir() if x.is_file()]
                frames = sorted(frames, key=lambda x: int(x.stem))
                for frame in frames[-15:]: # take only the last 15 frames
                    data = read_file(frame)
                    dist = calculate_distance(data)
                    samples.append(dist)
                    labels.append(label_map[action])
                    pers_id.append(p_id)
        p_id += 1
    return pers_id, samples, labels, 

actions = ['C','TL','TR','BL','BR']
idx, sam, lbl,  = read_data(actions)
print(f"Number of samples: {len(sam)} , and labels {len(lbl)}")



Number of samples: 7500 , and labels 7500


In [37]:
import pandas as pd

df = pd.DataFrame(sam, columns = connections_of_interest)
df['pers_id']=idx
df['class'] =  lbl
df.head()
df.columns

Index([   (0, 2),    (0, 5),    (0, 7),    (0, 8),    (0, 9),   (0, 10),
          (2, 5),    (2, 7),    (5, 8),    (7, 9),   (7, 11),   (8, 10),
         (8, 12),   (9, 10),   (9, 11),  (10, 12),  (11, 12),  (12, 10),
       'pers_id',   'class'],
      dtype='object')

# Training KNN

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.svm as svm
import cv2


In [9]:
X, y =read_data(actions)

In [10]:
print(f"Number of samples: {len(X)} , and labels {len(y)}")

Number of samples: 4750 , and labels 4750


In [11]:
C = 100.0  # SVM regularization parameter
models = (
    svm.SVC(kernel="linear", C=C),
    svm.LinearSVC(C=C, max_iter=10000, dual="auto"),
    svm.SVC(kernel="rbf", gamma=0.7, C=C),
    svm.SVC(kernel="poly", degree=3, gamma="auto", C=C),
)
models_fit = [clf.fit(X, y) for clf in models]



In [34]:
for model in models_fit:
    print(model)
    print("Training set score: {:.2f}".format(model.score(X_train, y_train)))
    print("Test set score: {:.2f}".format(model.score(X_test, y_test)))
    print()
    

SVC(C=100.0, kernel='linear')


NameError: name 'X_train' is not defined

# predict

In [12]:
model = models_fit[0]

In [13]:
file= Path(r"C:\Users\djase\Documents\Projecten\ML\ml_assignment_data\person_0000\BL\3\30.npy")
new_data = read_file(str(file))
new_sample= calculate_distance(new_data)

# Make predictions
predictions = model.predict([new_sample])

print(f"Predictions: {predictions}")

Predictions: [3]


In [14]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [15]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable =False
    results =  model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [16]:
def draw_custom_landmarks(image, results):
    if results.pose_landmarks:
    # Draw custom edges
        for connection in connections_of_interest:
            start_point = results.pose_landmarks.landmark[connection[0]]
            end_point = results.pose_landmarks.landmark[connection[1]]

            cv2.line(image, 
                     (int(start_point.x * image.shape[1]), int(start_point.y * image.shape[0])),
                     (int(end_point.x * image.shape[1]), int(end_point.y * image.shape[0])),
                     (255, 0, 0), 2) # Blue color, thickness 2

In [17]:
def predict_area(clf, results):
    pose =[[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark] if results.pose_landmarks else np.zeros(33*4)
    data = calculate_distance(pose)
    return clf.predict([data])

In [19]:
# webcam is max 1920 x 1080


cap = cv2.VideoCapture(0, cv2.CAP_DSHOW) 
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920/2)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080/2)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        #draw_styled_landmarks(image, results)
        draw_custom_landmarks(image, results)
        if results:
            label = predict_area(model, results)
            if label:
                cv2.putText(image, f'KIJKT NAAR {actions_description[actions[label[0]]]}', (120, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)

        cv2.imshow("Camera feed:", image)
        if cv2.waitKey(10) & 0xff == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

TypeError: 'numpy.float64' object is not iterable

In [20]:
cap.release()
cv2.destroyAllWindows()